In [34]:
# %%sh
# $HOME/.conda/envs/feat-seg-fusion/bin/python fusion.py --pipeline kmeans --pipeline_device cuda:1 --start 0 --end -1 --stride 1 --save_dir /home/quanta/Experiments/feature-instance-fusion/scannet_scene0000_00 --kmeans_cluster_num 512 --kmeans_extractor random_sam

In [35]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [36]:
import numpy as np
import open3d as o3d
import torch
from skimage import measure
from tqdm import tqdm, trange

from core.dataset import ScanNet
from core.integrate import PanopticFusionScalableTSDFVolume
from core.integrate.utils.misc import instance_id_to_one_hot_mask

In [37]:
tsdf_device = "cuda:1"
tsdf_volume = PanopticFusionScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device=tsdf_device,
)

save_dir = "/home/quanta/Experiments/feature-instance-fusion/scannet_scene0000_00/"
# save_dir = "/home/quanta/Experiments/feature-instance-fusion/scannet_scene0645_01/"
# save_dir = "/home/quanta/Experiments/feature-instance-fusion/scannet_scene0643_00/"
# save_dir = "/home/quanta/Experiments/feature-instance-fusion/scannet_scene0488_01/"
tsdf_volume.load(save_dir + "tsdf/tsdf_volume.pt")

In [38]:
verts, faces = tsdf_volume.extract_mesh()
mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(verts),
    triangles=o3d.utility.Vector3iVector(faces),
)

In [39]:
# load labels
voxel_labels = torch.load(save_dir + "kmeans_random_grounded_sam/kmeans_labels.pt").to(tsdf_device)
tsdf_volume.reset_instance()
tsdf_volume._instance_label_num = 1024
tsdf_volume._instance = voxel_labels

In [40]:
labels = tsdf_volume.extract_label_on_grid(verts=verts, device=tsdf_device)[0]

In [41]:
color_set = np.random.random(size=(1024, 3))

In [42]:
color = color_set[labels]
# mesh.compute_vertex_normals()
mesh.vertex_colors = o3d.utility.Vector3dVector(color)

In [43]:
import open3d as o3d

draw = o3d.visualization.EV.draw
draw([mesh])

In [44]:
o3d._build_config["ENABLE_HEADLESS_RENDERING"]

True